In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(1, parentdir+'/src')
import torch
import numpy as np
import time
from tqdm import trange
import matplotlib.pyplot as plt

In [3]:
import gym
import pybullet as p
import stage.envs
from stage.tasks.kuka import KukaPETS
from stage.utils.nn import use_gpu
use_gpu()

In [4]:
savepath = parentdir + '/data/kuka/'

In [5]:

kuka = KukaPETS(render=False, 
                   action_parameterization='pd')
nq, nv, nu, nx, na = kuka.nq, kuka.nv, kuka.nu, kuka.nx, kuka.na

In [ ]:
_ = kuka.learn(30, verbose=True)
kuka.save_training_data(savepath+'data_pd')

Network training: 100%|██████████| 20/20 [00:01<00:00, 11.64epoch(s)/s, Training loss MSE=1047.1971]


Iteration:  0
avg. decision time:  0.004517954587936401
obs. reward:  -5076.5137
act. reward:  -50.388203


Network training: 100%|██████████| 20/20 [00:03<00:00,  6.35epoch(s)/s, Training loss MSE=1030.4867]


Iteration:  1
avg. decision time:  0.31702234864234924
obs. reward:  -4316.46
act. reward:  -38.40787


In [ ]:
# traj, log = kuka.perform(kuka.goal)
# act_seq = traj[:, 14:28]
# initial_obs = traj[0, :14]
# final_obs = traj[-1, :14]

# n_sample = 50
# obs_traj = kuka.dynamics.unroll(initial_obs, 
#                                       act_seq, 
#                                       n_sample)

# predicted_err = torch.norm(obs_traj[-1, :, :]-final_obs.expand(n_sample, -1), p=2, dim=1)
# print (predicted_err.mean())
# print (predicted_err.std())

